# OTsafe

This is a demonstration of various use cases for OTsafe. 

## Demo

### Concept

During a recent crown jewel analysis, you decide that your next security priority is ensuring the safety of the personnel working around your equipment.  You decide that to keep people from being injured or killed by your cranes, that you'd like to explore using OTsafe to apply last-ditch safety countermeasures (BTW if you would like to test this IRL, contact me!). 

Think of OTsafe as an aftermarket safety system that can be built on top of new OR existing processes.  

OTsafe could conceptually create an additional SIS layer.  This would not be the perfect solution, but we owe it to these brave people who put themselves in harm's way.  

As with every problem, we need to distill down the key points and levers that we can control.  Inputs and outputs, that sort of thing.  

You realize that in order to prevent unsafe release of the clamps, we should cross check a couple of statuses.  We know these statements to be true:

- Before we unclamp the container, we should ensure we don't have tension on the line.  
- We also shouldn't be in motion. 

While we'e poking around, let's keep out eye open for more safety detection opportunities! 

### Background

Situation (true backstory but some details were changed):  

You run the OT cybersecurity program for a shipping port  company in Tampa.  Your company owns cargo ship docks and a several cranes, which they use to unload 20-foot and 40-foot shipping containers.  

A while back, someone in the parking lot joined the corporate Wi-Fi. Due to the flat network structure, the attacker was able to monitor the control system network traffic.  

The attacker eventually transmitted a specific Modbus packet which caused the crane's container clamp to open.  The contained dropped to the ground, the contents destroyed.  Luckily, no one was hurt!  Close call. 

On the bright side... you now have a budget!


### Capabilities

The following concepts will be demonstrated:

- Creation of individual components in OTsafe
- Establishing nominal, alarm, safe, and failure levels
- Modeling the relationship between multiple components
- Connect to network, PCAPs, historian, etc. 
- Simulate an industrial process w/ control loop
- Digital Lock Out Tag Out
- Demonstrate nominal operations
- Demonstrate safety detection engineering:
  - Simulate equipment failure
  - Simulate attacker in the network
  - Simulate supply chain attack
  - Automation (QA test and retroanalyze at the click of a button)
- Automation Hub:
  - Load inventory
  - Manage PCAP inventory
- Malware analysis (reconstructing intrusions)
- Audit Friendly
- Don't have to rely on or interpret ladder logic to understand safety mitigations
- Ties into SBOM, engineering, modeling, and safety disciplines
- Build encrypted tunnel on top of existing network traffic


## Control Loop Compromise and Detection

### About

"Control Loop Compromise" refers to the accidental or malicious influence of one or more components of a control loop, within a control system context.

This type of incident is not necessarily malicious.  A faulty valve is many more time more likely to cause an impact on the control loop.  

A more appropriat term could _almost_ be "integrity attack".   However, since control loop compromises could be accidental in nature, or due to equipment failure, this term is not 100% accurate.  Hence, "control loop compromise".  Propose a better name and let's talk about it!  


### Imports and Object Creation

In [ ]:
import pyshark

from otsafe.components.actuators import Actuator
from otsafe.components.sensors import Sensor
from otsafe.components.alarms import Alarm


cl_60 = Actuator(
    name = "Automatic clamp",
    description = "Clamp set used to securely grab a shipping container",
    id = "cl_60",
    status = "Open"
)

ls_12 = Sensor(
    name = "Load Sensor",
    description = "Crane Load Sensor reports the weight of the container",
    id = "ls_12",
    value = 0,
    units = "LBS"
)

So now we have two objects.  An Actuator (the clamp system), and the load sensor (measuring the weight of the load.)

I think you can see where this is going! 

In [ ]:
# Listen on local interface for Modbus packets

cap = pyshark.LiveCapture(interface='eth0')

for packet in cap.sniff_continuously(packet_count=5):

    # If the clamp is open and the load sensor is reporting a load, raise an alarm
    if cl_60.status == "Open" and ls_12.value > 0:
        cl_60.alarms.append(
            Alarm(
                name = "Load Detected",
                description = "Actuator opened while load detected",
                id = "ls_12",
                status = "Active"
          )
      )

This Alarm object that was created can be used to kick off curther automation.  We could use the included Redis image to build a subscriber who processes Alarm objects.  Or otherwise extend it yourself! 



# Future

Future possible use cases that OTsafe can support

- We could install additional hardware as time and budget allow, and continue to build safety detections based off of sensor data. 